E-Commerce

Week-1

In [1]:
#importing Data
import pandas as pd
import numpy as np

In [1]:
train_data=pd.read_csv("/kaggle/input/train_data.csv")

In [1]:
train_data.head(4)

In [1]:
train_data['reviews.text'][1:5]

In [1]:
train_data.columns

In [1]:
#Now Visualising Counts of Sentiment using Count Plot
import seaborn as sns
sns.set(style="darkgrid", font='sans-serif')
ax=sns.countplot(x="sentiment",data=train_data)

In [1]:
train_data[train_data['sentiment']=='Positive']['reviews.text'][4]

In [1]:
train_data[train_data['sentiment']=='Negative']['reviews.text'][97]

It is not possible that only very few neutral and Negative comments than Possitive hence data is
So we have to remove the imbalance of data

In [1]:
train_data['reviews.text'][1:5]

In [1]:
reviews=train_data['reviews.text'].values

Data Cleaning 

In [1]:
#Removing Of Stopwards,Panctuations,urls and # tags
import re

In [1]:
from nltk.tokenize import word_tokenize

In [1]:
text="Ram is very good fielder but he can not do well with bat"

In [1]:
text=word_tokenize(text)

In [1]:
print(text)

In [1]:
from nltk.corpus import stopwords

In [1]:
print(stopwords.words('english'))

In [1]:
stop_word=stopwords.words('english')

In [1]:
from string import punctuation

In [1]:
print(list(punctuation))

In [1]:
final_stop=stop_word+list(punctuation)+["...","``","====",'must']

In [1]:
print(reviews[1])

In [1]:
#Normalising the casing 
reviews=[word.lower() for word in reviews]

In [1]:
reviews[1]

In [1]:
#Tokenising the sentense into words
reviews=[word_tokenize(sent) for sent in reviews]

In [1]:
def stop(sent):
    return [word for word in sent if word not in final_stop]

In [1]:
reviews=[stop(sent) for sent in reviews]

In [1]:
reviews[2]

In [1]:
reviews=[" ".join(word) for word in reviews]

In [1]:
# Applying TF_IDF Vectoriser
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv=TfidfVectorizer(max_features=5000)

In [1]:
from sklearn.preprocessing import LabelEncoder
Le=LabelEncoder()

In [1]:
X=reviews

In [1]:
train_data['reviews']=X

In [1]:
y=Le.fit_transform(train_data['sentiment'])

In [1]:
train_data['target']=y

In [1]:
#Using train test split to split the data 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [1]:
x_sc_v=tfidfv.fit_transform(X_train)

In [1]:
x_test_sc=tfidfv.transform(X_test)

In [1]:
from sklearn.naive_bayes import MultinomialNB

In [1]:
Mlnb=MultinomialNB()

In [1]:
Mlnb.fit(x_sc_v,y_train)

In [1]:
y_pred=Mlnb.predict(x_sc_v)

In [1]:
y_pred

In [1]:
from sklearn.metrics import accuracy_score

In [1]:
y_v=tfidfv.fit_transform(train_data['sentiment'])

In [1]:
y_pred=pd.Series(y_pred)

In [1]:
y_train=pd.Series(y_train)

In [1]:
df=pd.DataFrame(data=y_pred,columns=['prediction'])

In [1]:
df['y_train']=y_train

In [1]:
df.head()

In [1]:
df.prediction.unique()

As we can see that 2=Positive Review due to imbalance in data set everything is converted into Positive as 
Model Ignored the neutral=1 and Negative=0

In [1]:
df.y_train.unique()

In [1]:
from sklearn.metrics import classification_report

In [1]:
print(classification_report(y_pred,y_train))

As we can see that f1_score is baised towards positive sentiments as classifier taken care of only majority
minority neglected here

In [1]:
y_test_pred=Mlnb.predict(x_test_sc)
y_test_pred

In [1]:
print(classification_report(y_test_pred,y_test))

Week-2

Tackling Class Imbalance Problem:

In [1]:
#Now We will Go through Unsampling and over sampling to deal with class imbalace


In [1]:
from sklearn.metrics import f1_score,roc_auc_score,roc_curve

Undersampling

In [1]:
minority_class_len_1=len(train_data[train_data['target']==1])
minority_class_len_1

In [1]:
minority_class_len_2c=len(train_data[train_data['target']==0])
minority_class_len_2c

In [1]:
majority_class_len_c=len(train_data[train_data['target']==2])
print(majority_class_len_c)

In [1]:
randomly_majority_indices_k=np.random.choice(majority_class_len_c,minority_class_len_2c,minority_class_len_1)
len(randomly_majority_indices_k)

In [1]:
minority_class_indices_1=train_data[train_data['target']==1].index
len(minority_class_indices_1)

In [1]:
minority_class_indices_2c=train_data[train_data['target']==0].index
len(minority_class_indices_2c)

In [1]:
under_sample_indices=np.concatenate([minority_class_indices_1,randomly_majority_indices_k,minority_class_indices_2c])

In [1]:
under_sample=train_data.loc[under_sample_indices]
under_sample.shape

In [1]:
tar='target'

In [1]:
sns.countplot(x=under_sample['target'],data=under_sample)

In [1]:
X=under_sample.reviews.values
y=under_sample.target.values

following all above steps again

In [1]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [1]:
tfidfv=TfidfVectorizer()
x_sc_v=tfidfv.fit_transform(X_train)

In [1]:
x_test_v=tfidfv.transform(X_test)

In [1]:
x_sc_v

In [1]:
Mlnb.fit(x_sc_v,y_train)

In [1]:
y_pred=Mlnb.predict(x_sc_v)

In [1]:
y_test_pred=Mlnb.predict(x_test_v)

In [1]:
print(classification_report(y_test_pred,y_test))

In [1]:
y_test_proba=Mlnb.predict_proba(x_test_v)

In [1]:
roc_auc_score(y_test_pred,y_test_proba,multi_class='ovo')

In [1]:
print(classification_report(y_test_pred,y_test))

From Above we can see that by using under sampling we have some balanced f1_score but still it is 
very less so we have to improve f1_score for this we have to try another classification model

In [1]:
from sklearn.ensemble import RandomForestClassifier

In [1]:
Random_Forest=RandomForestClassifier()

In [1]:
X=reviews
y=train_data.target.values

In [1]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [1]:
#Training Data
X_train_v=tfidfv.fit_transform(X_train)
x_test_v=tfidfv.transform(X_test)

In [1]:
Random_Forest.fit(X_train_v,y_train)

In [1]:
y_pred_train=Random_Forest.predict(X_train_v)
y_pred_test=Random_Forest.predict(x_test_v)

In [1]:
#Training Data
print(classification_report(y_pred_train,y_train))

In [1]:
#Testing Data
print(classification_report(y_pred_test,y_test))

In [1]:
y_test_proba=Random_Forest.predict_proba(x_test_v)
roc_auc_score(y_test,y_test_proba,multi_class='ovr')

as we can see that training accuracy is good but when go to test accuracy get decreased

As we can see that model is giving good accuracy in training but giving less accuracy in testing so model is either subjected over fitting or model is not well responging to new data so for that reason we can perform 
hyper-parameter tuning 

In [1]:
from sklearn.model_selection import GridSearchCV,StratifiedKFold

In [1]:
n_estimator=[int(x) for x in np.linspace(start =1, stop = 1000, num = 10)]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]

In [1]:
param_grid={ 'max_depth':max_depth,'n_estimators':n_estimator}

In [1]:
Grid=GridSearchCV(estimator=Random_Forest,param_grid=param_grid,cv=StratifiedKFold(4),n_jobs=-1,verbose=2)

In [1]:
Grid.fit(X_train_v,y_train)

In [1]:
y_grid_train=Grid.predict(X_train_v)

In [1]:
print(classification_report(y_train,y_grid_train))

In [1]:
import pickle
Pkl_Filename = "Pickle_RL_Model.pkl"  

In [1]:
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(Grid, file)

In [1]:
y_grid_test=Grid.predict(x_test_v)

In [1]:
print(classification_report(y_test,y_grid_test))

In [1]:
Grid.best_estimator_.fit(X_train_v,y_train)

In [1]:
Random_Forest=RandomForestClassifier(max_depth=60,n_estimators=112)

In [1]:
Random_Forest.fit(X_train_v,y_train)

In [1]:
y_test_pred=Random_Forest.predict(x_test_v)

In [1]:
print(classification_report(y_test_pred,y_test))